In [ ]:
from insult_filter_single import insult_filter_single
import lithops
import boto3
import time

def stream(func, maxfunc, queue_url):
    fexec = lithops.FunctionExecutor()
    sqs = boto3.client('sqs', region_name='us-east-1')

    futures = []

    print("🚀 Iniciando procesamiento continuo...")

    while True:
        # Obtener número de mensajes pendientes
        attributes = sqs.get_queue_attributes(
            QueueUrl=queue_url,
            AttributeNames=['ApproximateNumberOfMessages']
        )
        num_msgs = int(attributes['Attributes']['ApproximateNumberOfMessages'])

        # Salir si ya no hay mensajes ni funciones activas
        if num_msgs == 0 and not any(not f.done for f in futures):
            print("✔️ Cola vacía y procesamiento completado.")
            break

        # Lanzar nuevas funciones si hay espacio
        while len(futures) < maxfunc and num_msgs > 0:
            future = fexec.call_async(func, None)
            futures.append(future)
            num_msgs -= 1
            print(f"→ Función lanzada ({len(futures)}/{maxfunc})")

        # Mostrar resultados de funciones completadas
        for f in futures:
            if f.ready and not hasattr(f, "_printed"):
                print("✅", f.result())
                f._printed = True

        # Eliminar funciones que ya han terminado
        futures = [f for f in futures if not f.ready]
        time.sleep(0.5)


In [ ]:
queue_url = 'https://sqs.us-east-1.amazonaws.com/164573654608/InsultTextQueue'
stream(insult_filter_single, maxfunc=3, queue_url=queue_url)